In [1]:
import pandas as pd
import networkx as nx

In [2]:
df = pd.read_csv("covid_spread.csv")

In [3]:
G = nx.DiGraph()
G.add_edges_from(zip(df['source'], df['destination']))

In [4]:
properties = {}

In [5]:
label = {}
for node in G.nodes():
    label[node] = node
properties['label'] = label

In [6]:
def compute_eccentricity(G):
    eccentricity = {}
    for node in G.nodes():
        try:
            eccentricity[node] = max(nx.shortest_path_length(G, source=node).values())
        except nx.NetworkXNoPath:
            eccentricity[node] = float('inf')
    return eccentricity

properties['eccentricity'] = compute_eccentricity(G)

In [7]:
def closeness_centrality(G):
    closeness = {}
    n = len(G)
    for node in G:
        distances = nx.shortest_path_length(G, source=node)
        total_distance = sum(distances.values())
        if total_distance > 0 and len(distances) > 1:
            closeness[node] = (len(distances) - 1) / total_distance
        else:
            closeness[node] = 0
    return closeness

properties['closenesscentrality'] = closeness_centrality(G)

In [8]:
from collections import deque

def betweenness_centrality(G):
    betweenness = {node: 0 for node in G.nodes}
    nodes = list(G.nodes)
    for source in nodes:
        stack = []
        predecessors = {node: [] for node in G.nodes}
        shortest_paths = {node: 0 for node in G.nodes}
        shortest_paths[source] = 1
        distances = {node: -1 for node in G.nodes}
        distances[source] = 0
        queue = deque([source])
        while queue:
            v = queue.popleft()
            stack.append(v)
            for neighbor in G.neighbors(v):
                if distances[neighbor] < 0:
                    queue.append(neighbor)
                    distances[neighbor] = distances[v] + 1
                if distances[neighbor] == distances[v] + 1:
                    shortest_paths[neighbor] += shortest_paths[v]
                    predecessors[neighbor].append(v)
        dependency = {node: 0 for node in G.nodes}
        while stack:
            w = stack.pop()
            for v in predecessors[w]:
                dependency[v] += (shortest_paths[v] / shortest_paths[w]) * (1 + dependency[w])
            if w != source:
                betweenness[w] += dependency[w]
    for node in betweenness:
        betweenness[node] /= 2
    return betweenness

properties['betweennesscentrality'] = betweenness_centrality(G)

In [9]:
import community as community_louvain
modularity_classes = community_louvain.best_partition(nx.Graph(G))
properties['modularity_class'] = modularity_classes

In [10]:
def pagerank(G, alpha=0.85, max_iter=100, tol=1e-6):
    nodes = list(G.nodes)
    n = len(nodes)
    pagerank = {node: 1 / n for node in nodes}
    for _ in range(max_iter):
        new_pagerank = {}
        for node in nodes:
            rank_sum = sum(pagerank[neighbor] / len(list(G.neighbors(neighbor))) for neighbor in G.predecessors(node))
            new_pagerank[node] = (1 - alpha) / n + alpha * rank_sum
        if all(abs(new_pagerank[node] - pagerank[node]) < tol for node in nodes):
            break
        pagerank = new_pagerank
    return pagerank

properties['pagerank'] = pagerank(G)

In [11]:
import numpy as np

def eigen_centrality(G, max_iter=100, tol=1e-6):
    nodes = list(G.nodes)
    n = len(nodes)
    index = {node: i for i, node in enumerate(nodes)}
    adjacency_matrix = np.zeros((n, n))
    for node, neighbors in G.adjacency():
        for neighbor in neighbors:
            adjacency_matrix[index[node], index[neighbor]] = 1

    eigenvector = np.ones(n)/n
    for _ in range(max_iter):
        next_eigenvector = adjacency_matrix @ eigenvector
        next_eigenvector /= np.linalg.norm(next_eigenvector,2)
        if np.linalg.norm(next_eigenvector - eigenvector,2) < tol:
            break
        eigenvector = next_eigenvector

    return {nodes[i]: eigenvector[i] for i in range(n)}

properties['eigencentrality'] = eigen_centrality(G)

In [12]:
properties['indegree'] = dict(G.in_degree())
properties['outdegree'] = dict(G.out_degree())

In [13]:
properties['degree'] = dict(G.degree())

In [14]:
result_df = pd.DataFrame(properties)

In [15]:
result_df

,label,eccentricity,closenesscentrality,betweennesscentrality,modularity_class,pagerank,eigencentrality,indegree,outdegree,degree
China,China,3,0.541667,3715.495694,0,0.004889,0.402738,9,26,35
Thailand,Thailand,4,0.396610,0.393995,0,0.001208,0.143563,2,10,12
Vietnam,Vietnam,4,0.360000,13.009524,0,0.001360,0.063377,2,3,5
Japan,Japan,4,0.413428,160.729333,0,0.002286,0.217008,6,13,19
United States,United States,4,0.427007,2771.750560,1,0.032046,0.220532,56,30,86
...,...,...,...,...,...,...,...,...,...,...
Saint Kitts and Nevis,Saint Kitts and Nevis,5,0.301020,0.000000,1,0.000898,0.026872,0,1,1
Slovakia,Slovakia,1,1.000000,0.000000,6,0.000898,0.000000,0,1,1
Maldives,Maldives,1,1.000000,0.000000,6,0.000898,0.000000,0,1,1
Cape Verde,Cape Verde,1,1.000000,0.000000,6,0.000898,0.000000,0,1,1


In [16]:
result_df.to_csv("node_properties.csv", index_label="node")